<a href="https://colab.research.google.com/github/linked0/merkle-root/blob/master/python/merkle_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Jupyter notebook for protyping and testing in Python 3.x environment

In [52]:
import sys, struct
from hashlib import sha256
# import binascii

class MerkleTree(object):
    def __init__(self, values):
        self.leaf_data = values
        self.root_node = None
        
        # to make full balanced tree, find nearest multiples of 2 above the length
        leaf_count = 0
        if len(self.leaf_data) > 1:
            leaf_count = 1
            while leaf_count < len(self.leaf_data):
                leaf_count *= 2
        elif len(self.leaf_data) == 1:
            leaf_count = 1
        
        # the last value is copied to empty slots in list
        for i in range(len(values), leaf_count):
            self.leaf_data.append(self.leaf_data[-1])

    def make_tree(self):
        current_level = []
        current_node = None
        
        # get hash value for data and save the value to node structure
        for data in self.leaf_data:
            hash_value = self.get_double_sha256(data.encode('ascii'), None)
            current_node = dict({'hash': hash_value, 'data': data, 'child': None})
#             print(self.convert_binary_to_ascii(current_node['hash']))
            current_level.append(current_node)

        # making non-leaf nodes continues until only one node exists
        iter_count = 0
        while len(current_level) != 1:
            parent_level = []
            for i in range(0, len(current_level), 2):
                node1 = current_level[i]
                node2 = current_level[i+1]
                hash_value = self.get_double_sha256(node1['hash'], node2['hash'])
                current_node = dict({'hash': hash_value, 'data': None, 'child': [node1, node2]})
                parent_level.append(current_node)

            iter_count += 1
            current_level = parent_level

        self.root_node = current_node

    def get_double_sha256(self, value1, value2):\
        # this function uses sha256 functions of the hashlib package
        m = sha256()
        m.update(value1)
        if value2 != None:
            m.update(value2)
        one_hash = m.digest()
        
        # date is double-sha256 hashed
        double_hash = sha256(one_hash).digest()
        return double_hash

    def print_tree(self, nodes, level=0):
        if nodes == None:
            nodes = [self.root_node]

        print('Level {0}:'.format(level))
        next_print_nodes = []
        for node in nodes:
            print(self.convert_binary_to_ascii(node['hash']))
            if node['child'] != None:
                next_print_nodes = next_print_nodes + node['child']

        if len(next_print_nodes) != 0:
            self.print_tree(next_print_nodes, level+1)
        
    def convert_binary_to_ascii(self, hash_str):
        # this function converts a binary string to ascii string
        ascii_str = ''
        chunk_size = 8
        for i in range(0, len(hash_str), chunk_size):
            bin_str = hash_str[i:i+chunk_size]
            res = struct.unpack('<BBBBBBBB', bin_str)
            hex_str = ''.join('%02X%02X%02X%02X%02X%02X%02X%02X'%(res))
            ascii_str += hex_str
        return ascii_str
    
#     def convert_binary_to_ascii_with_binascii(self, hash):
#         if sys.version_info < (3,0,0):
#             hash_str = str(binascii.hexlify(hash))
#             return hash_str.encode(encoding='ascii').upper()
#         else:
#             return str(binascii.hexlify(hash), 'ascii').upper()

            
def merkle(argv):
    mt = MerkleTree(argv)
    mt.make_tree()
    mt.print_tree(None)
  
  
merkle(['The quick brown fox', 'jump over', 'the', 'lazy dog'])
print('======================')
merkle(['The quick brown fox', 'jump over', 'the', 'lazy', 'dog'])
print('======================')
merkle([''])
print('======================')
merkle(['', ''])

Level 0:
018FB04252A594A8049CBFE9E34848249040E1FA7E170501E17ADC06393D4DC3
Level 1:
836C2FE675884DB41C49215F8A91E6560B1EA752F683DB793E9B86180CA235F8
49B9A6B1346DC768898A16C2DAD9D554349C9150F8B2809AC7D48B305C4D3650
Level 2:
7743034D22491720B723B68AFD046BE66969409254DC79A153E290C81A8F238A
F51DF418D9D7BAFDCFDC4320409E08E39858D0D686FEE959EA545E6D7C214F71
1E7C521A055F0F08CEA3FADED5923CCA2D8F4366A62AAA8A8B843A842AA656B8
144BEE93D8F6350C6E38C96EEB11DE2CD249A7BD5D23FF4C91EB46573B5AF3BA
Level 0:
9AF409C11D320898DA335F82FAC8918014A6589E55F2C98F2B3C468ED83F6ACE
Level 1:
6E76CEB1CBD48BE752949D832B4AB2848E7B9DF6A4286A524C7B134A4A9DF458
156B626C730889C1CD1A1F05D55EC3C16AF6DC5BD6977E0BE2A1935C86A74CCF
Level 2:
836C2FE675884DB41C49215F8A91E6560B1EA752F683DB793E9B86180CA235F8
31A1892ED0C9857597D9D8902E1A05D5A3A99DEAD7FADB9E69BF319FC22C1AE6
CBBC43D5431E285C2770E9F9084663CB6CCF4B23228DD8E90A30B361A80CC57D
CBBC43D5431E285C2770E9F9084663CB6CCF4B23228DD8E90A30B361A80CC57D
Level 3:
7743034D22491720B723B68AFD0